## Input files

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/gdrive')
filepath = "/gdrive/My Drive/files/projects/bir_zonal_values/data/"

Mounted at /gdrive


In [ ]:
rdo_region = pd.read_csv(f"{filepath}rdo_region.csv")
rdo_region.head(2)

,RDO,Region
0,RDO No. 1 - Laoag City,"Revenue Region 1 - Calasiao, Pangasinan"
1,"RDO No. 2 - Bantay, Ilocos Sur","Revenue Region 1 - Calasiao, Pangasinan"


In [ ]:
directory = pd.read_csv(f"{filepath}directory.csv")
directory.head(2)

,Unnamed: 0,RDO,Region,rdonum,filename
0,0,RDO No. 1 - Laoag City,"Revenue Region 1 - Calasiao, Pangasinan",1,"bir_zonal_values/RDO No. 1 - Laoag City, Iloco..."
1,1,"RDO No. 2 - Bantay, Ilocos Sur","Revenue Region 1 - Calasiao, Pangasinan",2,"bir_zonal_values/RDO No. 2 - Bantay, Ilocos Su..."


In [ ]:
regions = directory["Region"].unique().tolist()
regions[0:2]

['Revenue Region 1 - Calasiao, Pangasinan',
 'Revenue Region 2 - CAR (Cordillera Administrative Region)']

In [ ]:
filename = "bir_zonal_values.zip"
!gsutil cp "{filepath}{filename}" /content/
!unzip {filename}

Copying file:///gdrive/My Drive/files/projects/bir_zonal_values/data/bir_zonal_values.zip...
-
Operation completed over 1 objects/49.6 MiB.                                     
Archive:  bir_zonal_values.zip
   creating: bir_zonal_values/
  inflating: bir_zonal_values/RDO No. 52 - Paranaque City.xls  
  inflating: bir_zonal_values/RDO No. 72 - Roxas City, Capiz.xls  
  inflating: bir_zonal_values/RDO No. 71 - Kalibo, Aklan.xls  
  inflating: bir_zonal_values/RDO No. 74 - Iloilo City, Iloilo.xls  
  inflating: bir_zonal_values/RDO No. 13 - Tuguegarao City.xls  
  inflating: bir_zonal_values/RDO No. 69 - Virac, Catanduanes.xls  
  inflating: bir_zonal_values/RDO No. 21B - City of San Fernando, South Pampanga.xls  
  inflating: bir_zonal_values/RDO No. 88 - Tacloban City.xls  
  inflating: bir_zonal_values/RDO No. 75 - Zarraga, Iloilo.xls  
  inflating: bir_zonal_values/RDO No. 60 - Lucena City, North Quezon.xls  
  inflating: bir_zonal_values/RDO No. 107 - Cotabato City.xls  
  inflating

In [ ]:
filename = "preprocessed1.zip"
!gsutil cp "{filepath}{filename}" /content/
!unzip {filename}

Copying file:///gdrive/My Drive/files/projects/bir_zonal_values/data/preprocessed1.zip...
-
Operation completed over 1 objects/12.8 MiB.                                     
Archive:  preprocessed1.zip
   creating: preprocessed/
  inflating: preprocessed/Revenue Region 13 - Cebu City.csv  
  inflating: preprocessed/Revenue Region 8B - South NCR.csv  
  inflating: preprocessed/Revenue Region 7A - Quezon City.csv  
  inflating: preprocessed/Revenue Region 14 - Tacloban City.csv  
  inflating: preprocessed/Revenue Region 16 - Cagayan De Oro City.csv  
  inflating: preprocessed/Revenue Region 9B - LAQUEMAR (Laguna - Quezon - Marinduque).csv  
  inflating: preprocessed/Revenue Region 19 - Davao City.csv  
  inflating: preprocessed/Revenue Region 8A - Makati City.csv  
  inflating: preprocessed/Revenue Region 1 - Calasiao, Pangasinan.csv  
  inflating: preprocessed/Revenue Region 5 - Caloocan.csv  
  inflating: preprocessed/Revenue Region 11 - Iloilo City.csv  
  inflating: preprocessed/Reve

## Get excel file per RDO

In [ ]:
# Google Colab
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# Fetch the html file
url = "https://www.bir.gov.ph/index.php/zonal-values.html"
r = requests.get(url)
html_doc = r.text

soup = BeautifulSoup(html_doc, 'html.parser')

matches = soup.find_all("a", string="Zip Excel File")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
for m in tqdm(matches):
    !wget https://www.bir.gov.ph{m['href']}

In [ ]:
len(matches)
from glob import glob
downloads = glob('*.zip')
len(downloads)
!mkdir bir_zonal_values
for _file in tqdm(downloads):
    !unzip -d bir_zonal_values "{_file}"

120

In [ ]:
!zip -r bir_zonal_values.zip bir_zonal_values

In [ ]:
!unzip bir_zonal_values.zip

## Group by Revenue Region

In [ ]:
from glob import glob
files = glob("bir_zonal_values/*")
files[0:2]

['bir_zonal_values/RDO No. 52 - Paranaque City.xls',
 'bir_zonal_values/RDO No. 72 - Roxas City, Capiz.xls']

In [ ]:
files.sort()

In [ ]:
len(files)

122

In [ ]:
df = rdo_region.copy()

def get_rdonum(text):
    "output: '55'"
    return (
        text
        .split("-")[0]
        .split('/')[-1]
        .split(".")[-1]
        .strip()
    )

_dict = {}
for f in files:
    rdonum = get_rdonum(f)
    _dict[rdonum] = f

def get_filename(rdonum):
    try:
        return _dict[str(rdonum)]
    except:
        return ""

df["rdonum"] = df["RDO"].apply(get_rdonum)
df["filename"] = df["rdonum"].apply(get_filename)

df.head(2)

,RDO,Region,rdonum,filename
0,RDO No. 1 - Laoag City,"Revenue Region 1 - Calasiao, Pangasinan",1,"bir_zonal_values/RDO No. 1 - Laoag City, Iloco..."
1,"RDO No. 2 - Bantay, Ilocos Sur","Revenue Region 1 - Calasiao, Pangasinan",2,"bir_zonal_values/RDO No. 2 - Bantay, Ilocos Su..."


In [ ]:
df.to_csv("directory.csv")

In [ ]:
# !pip install xlrd==1.2.0
import pandas as pd
from tqdm import tqdm
import numpy as np

def add_repeating_column(df,label):
    df.insert(loc=0, column='00', value=label)
    return df

def unname_columns(df):
    "columns become 0,1,2,3"
    df.columns = range(len(df.columns))
    return df

def get_rdonum(text):
    "output: '55'"
    return (
        text
        .split("-")[0]
        .split('/')[-1]
        .split(".")[-1]
        .strip()
    )

def collect_sheets(excel_file):
    sheets = pd.read_excel(excel_file,sheet_name=None)
    dfs = []
    rdonum = get_rdonum(excel_file)
    print(f"Collecting RDO No. {rdonum}")
    for k, v in sheets.items():
        df = v
        df2 = add_repeating_column(df,k)
        df3 = unname_columns(df2)
        dfs.append(df3)
    df4 = pd.concat(dfs, axis=0)
    df5 = add_repeating_column(df4, rdonum)
    df6 = unname_columns(df5)
    return df5

def collect_region(directory, region):
    print(f"Collecting {region}")
    sub = directory.query(f"Region == '{region}'")
    files = sub.filename.unique().tolist()
    dfs = []
    
    for f in tqdm(files):
        dfs.append(collect_sheets(f))
    df = pd.concat(dfs, axis=0)
    df2 = add_repeating_column(df, region)
    df3 = unname_columns(df2)
    return df3


In [ ]:
df = collect_sheets('bir_zonal_values/RDO No. 12 - Lagawe, Ifugao.xlsx')

In [ ]:
df.to_csv('collected.csv')

In [ ]:
directory = pd.read_csv("directory.csv")
regions = directory["Region"].unique().tolist()

In [ ]:
!mkdir revenue_regions/

In [ ]:
for region in tqdm(regions):
    df = collect_region(directory,region)
    df.to_csv(f"revenue_regions/{region}.csv")

  0%|          | 0/22 [00:00<?, ?it/s]


 17%|█▋        | 1/6 [00:02<00:12,  2.50s/it]


 33%|███▎      | 2/6 [00:03<00:06,  1.58s/it]


 50%|█████     | 3/6 [00:06<00:07,  2.36s/it]


 67%|██████▋   | 4/6 [00:08<00:04,  2.04s/it]

...


 40%|████      | 2/5 [00:16<00:21,  7.32s/it]


 60%|██████    | 3/5 [00:17<00:08,  4.13s/it]


 80%|████████  | 4/5 [00:17<00:02,  2.63s/it]


100%|██████████| 5/5 [00:18<00:00,  3.65s/it]


100%|██████████| 22/22 [01:58<00:00,  5.37s/it]


In [ ]:
!zip -r revenue_regions.zip revenue_regions

  adding: revenue_regions/ (stored 0%)
  adding: revenue_regions/Revenue Region 4 - San Fernando, Pampanga.csv (deflated 92%)
  adding: revenue_regions/Revenue Region 16 - Cagayan De Oro City.csv (deflated 92%)
  adding: revenue_regions/Revenue Region 8A - Makati City.csv (deflated 87%)
  adding: revenue_regions/Revenue Region 9B - LAQUEMAR (Laguna - Quezon - Marinduque).csv (deflated 93%)
  adding: revenue_regions/Revenue Region 7B - East NCR.csv (deflated 87%)
  adding: revenue_regions/Revenue Region 8B - South NCR.csv (deflated 89%)
  adding: revenue_regions/Revenue Region 5 - Caloocan.csv (deflated 90%)
  adding: revenue_regions/Revenue Region 6 - Manila.csv (deflated 97%)
  adding: revenue_regions/Revenue Region 2 - CAR (Cordillera Administrative Region).csv (deflated 93%)
  adding: revenue_regions/Revenue Region 12 - Bacolod City.csv (deflated 94%)
  adding: revenue_regions/Revenue Region 3 - Tuguegarao, Cagayan.csv (deflated 93%)
  adding: revenue_regions/Revenue Region 14 - Tac

## Filter to relevant columns then rows

In [ ]:
!unzip revenue_regions.zip

In [ ]:
!mkdir preprocessed/

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
import re

place_types = [
    "province","barangay",
    "city","municipality",
    # ,"ity" # city or municipality
]

def check_valid_zv(text):
    # check if text is just made up of numbers and asterisks
    if text is not None:
        if len(re.sub("[0-9\*]","",text))==0:
            return True
    return False

def check_do_no(text):
    if text is not None:
        if re.search("d.o. no",text.lower()):
            return True
    return False

def check_effectivity_date(text):
    if text is not None:
        if re.search("effectivity",text.lower()):
            return True
    return False

def clean_date(_row):
    row = _row.fillna("")
    
    # if there's at least 1 number, it's probably a valid date
    if re.search("[0-9]",row["effectivity_date"]):
        _date = row["effectivity_date"]
    else:
        # get it from the sheet name
        _date ="12-31-" + str(row["sheet_name"].split("-")[-1][:-2])
    
    try:
        return pd.to_datetime(_date)
    except:
        # fix for 12-31-202 in Revenue Region 8A - Makati City
        # print(f"Warning: invalid date at row{row['index']}")
        return None

    

def check_placename(row,col_num=4):
    row_as_text = str(row[col_num]).lower()
    if row_as_text is not None:
        if len(row_as_text) < 50: # if long probably a paragraph text
            # if ":" in row_as_text:
            if re.search("province|city|municipality|barangay",row_as_text):
                return True
    return False

def get_place_names(row,start_col=4,end_col=5):
    row_as_text = (
        re.sub(' +', ' ', # dedup spaces
        " ".join(row.fillna("").tolist()[start_col:end_col+1]) # combine columns to 1 text
        ).lower()
    )
    place_name = row_as_text.split(":")[-1].strip()
    for place_type in place_types:
        if re.search(place_type,row_as_text):
            return place_type, place_name
    return "province", ""

In [ ]:
# why use classes vs plain functions for data ingestion
# - state dependent - not interchangeable steps (i.e. step1 must happen before step2)
# - heavy coupling - scattered functions, not loosely coupled column operations
# - inconsistent interfaces - output of parts of code not the same as others, expected functions non existent

# class Processor():
#     def init(df):
#         self.df = df
#     def process():
#         assert len(self.df) > 0
#         return self.df

In [ ]:
# region = 'Revenue Region 8A - Makati City' #regions[5]
# out_dir="preprocessed/"

def preprocess(region,out_dir="preprocessed/"):
    print(f"Processing {region}")
    df = pd.read_csv(f"revenue_regions/{region}.csv")
    window = df.iloc[:,1:8].reset_index()
    window["has_placename"] = False
    window["has_do_no"] = False

    print("Getting rows with place names")
    recs = []
    for i, row in tqdm(window.iterrows(),total=len(window)):
        recs.append(check_placename(row))
    window["has_placename"] = recs
    placenames = window[window["has_placename"]]

    print("Getting place names from the rows")
    placenames2 = placenames.copy()
    recs = {
        '1': [], 
        '2': []
    }
    for i, row in tqdm(placenames2.iterrows(),total=len(placenames2)):
        place_type, place_name = get_place_names(row)
        recs['1'].append(place_type)
        recs['2'].append(place_name)
    placenames2["place_type"] = recs['1']
    placenames2["place_name"] = recs['2']
    pivoted = placenames2.pivot(index="index", columns="place_type", values="place_name")
    if "municipality" not in pivoted.columns:
        # fixes error for RDO 6 Manila
        pivoted["municipality"] = None
    placenames3 = pd.concat([placenames2,pivoted],axis=1)

    print("Finding rows with zonal values")
    valid_zvs = window.loc[window['6'].astype(str).apply(check_valid_zv),:]

    print("Finding rows with effectivity dates")
    effectivity_dates = window.loc[window['5'].astype(str).apply(check_effectivity_date),:]
    effectivity_dates["effectivity_date"] = window['6']

    print("Finding rows with D.O. No.")
    do_no = window.loc[window['5'].astype(str).apply(check_do_no),:]
    do_no["D.O. No"] = window['6']
    do_no["has_do_no"] = True


    print("Combining to 1 table")
    output = (
        pd.concat([placenames3,do_no,effectivity_dates,valid_zvs],axis=0)
        .sort_values("index")
        .replace(r'^\s*$', np.nan, regex=True) # replace your empty cells with NaNs
        .fillna(method='ffill')
        .query("~barangay.isnull()", engine='python')
        .query("has_do_no == False")
        .query("has_placename == False")
        .rename(columns={
            "0":"revenue_region","1":"rdonum","2":"sheet_name",
            "3":"street_subdivision","4":"vicinity","5":"classification","6":"zv_per_sqm"
        })
    )

    print("Adding source filename")
    output["rdonum"] = output["rdonum"].astype(str)
    joined = pd.merge(output,directory[['RDO','rdonum']],on='rdonum')

    print("Cleaning dates")
    recs = []
    for i, row in tqdm(joined.iterrows(), total=len(joined)):
        # fixes error for RR 1 Calasiao where effectivity_date = "Effectivity Date"
        recs.append(clean_date(row))

    joined["effectivity_date"] = recs


    print("Reordering columns")
    cols = [
        "index",
        "revenue_region",
        "RDO",
        "sheet_name",
        "street_subdivision",
        "vicinity",
        "classification",
        "zv_per_sqm",
        "D.O. No",
        "effectivity_date",
        "rdonum",
        "barangay",
        "city",
        "municipality",
        "province",
        "has_placename",
        "has_do_no",
        "place_type",
        "place_name",
    ]

    print("Saving output file")
    joined[cols].to_csv(f"{out_dir}{region}.csv",index=False)

    return joined[cols]

In [ ]:
_dict = {}
for region in tqdm(regions):
    df = preprocess(region)
    _dict[region] = len(df)

  0%|          | 0/22 [00:00<?, ?it/s]

Processing Revenue Region 1 - Calasiao, Pangasinan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (8,9,10,11,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


Getting rows with place names



100%|██████████| 211883/211883 [00:11<00:00, 17880.19it/s]


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Combining to 1 table
Adding source filename
Cleaning dates



100%|██████████| 57228/57228 [00:39<00:00, 1456.34it/s]


Reordering columns
Saving output file


100%|██████████| 22/22 [19:01<00:00, 51.89s/it]


In [ ]:
!zip -r preprocessed1.zip preprocessed

updating: preprocessed/ (stored 0%)
updating: preprocessed/Revenue Region 13 - Cebu City.csv (deflated 94%)
updating: preprocessed/Revenue Region 8B - South NCR.csv (deflated 95%)
updating: preprocessed/Revenue Region 7A - Quezon City.csv (deflated 94%)
updating: preprocessed/Revenue Region 14 - Tacloban City.csv (deflated 96%)
updating: preprocessed/Revenue Region 16 - Cagayan De Oro City.csv (deflated 97%)
updating: preprocessed/Revenue Region 9B - LAQUEMAR (Laguna - Quezon - Marinduque).csv (deflated 96%)
updating: preprocessed/Revenue Region 19 - Davao City.csv (deflated 96%)
updating: preprocessed/Revenue Region 8A - Makati City.csv (deflated 95%)
updating: preprocessed/Revenue Region 1 - Calasiao, Pangasinan.csv (deflated 96%)
updating: preprocessed/Revenue Region 5 - Caloocan.csv (deflated 95%)
updating: preprocessed/Revenue Region 11 - Iloilo City.csv (deflated 95%)
updating: preprocessed/Revenue Region 6 - Manila.csv (deflated 95%)
updating: preprocessed/Revenue Region 18 - Ko

## Check completeness

In [ ]:
from tqdm import tqdm
import pandas as pd
import xlrd

In [ ]:
from glob import glob
files = glob("bir_zonal_values/*")
files[0:2]

['bir_zonal_values/RDO No. 106 - Tandag City, Surigao del Sur.xls',
 'bir_zonal_values/RDO No. 115 - Digos City, Davao del Sur.XLS']

In [ ]:
dfs = []
for f in tqdm(files):
    book = xlrd.open_workbook(f)
    dfs.append(pd.DataFrame({"filename":f,"sheet_name":book.sheet_names()}))

raw_sheetnames = pd.concat(dfs,axis=0)

100%|██████████| 122/122 [00:46<00:00,  2.64it/s]


In [ ]:
_dict = {}
dfs = []
for region in tqdm(regions):
    df = pd.read_csv(f"preprocessed/{region}.csv")
    count_subset = (df[["revenue_region","RDO","sheet_name","vicinity"]]
                    .groupby(["revenue_region","RDO","sheet_name"]).count().reset_index())
    dfs.append(count_subset)
    _dict[region] = len(df)
preprocessed_sheetnames = pd.concat(dfs)

 23%|██▎       | 5/22 [00:01<00:03,  4.99it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 45%|████▌     | 10/22 [00:01<00:01,  8.48it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 22/22 [00:03<00:00,  5.78it/s]


In [ ]:
missing = list(set(raw_sheetnames.sheet_name).difference(set(preprocessed_sheetnames.sheet_name)))
missing

['Sheet 2 (DO 25-14)',
 'Sheet 3 (DO 12-2018)',
 'Sheet4 (DO 74-18',
 'Sheet3 (DO 059-18)',
 'Sheet 4 (DO 40-2020)',
 'Sheet1',
 'Recovered_Sheet4',
 'Recovered_Sheet3',
 'Sheet 1 (DO 52-95)',
 'Sheet 2 (DO 038-2014)',
 'Recovered_Sheet1',
 'Sheet 7 (DO 008-2020)',
 'Sheet 6 (DO 086-14)',
 'Sheet 1 (DO 17-89)',
 'Sheet 3 (DO 71-2015)',
 'pldt',
 'Sheet 6 (DO 21-2015)',
 'Sheet 3 (DO 12-15)',
 'Sheet 5 (DO 41-03)',
 'Sheet6 (DO 60-2018)',
 'Recovered_Sheet2',
 'Sheet 2 (DO 61-02)',
 'Sheet 1 (DO 79-94)',
 'Sheet 2 (DO 20-02)',
 'Sheet 3 (DO 63-17)',
 'Sheet 10 (DO 20-2020',
 'Sheet 4 (DO 14-2021)',
 'Compatibility Report',
 'Sheet 6 (DO 073-2018)',
 'Sheet 8 (DO 056-2019)',
 'Sheet 9 (DO 14-18)',
 ' Sheet 9 (DO 88-19)',
 'Recovered_Sheet5',
 'Sheet 6 (DO 75-17)',
 'Sheet 7 (DO 06-19)',
 'Sheet 6 (DO 54-97)',
 'Recovered_Sheet6',
 'Sheet 6 (DO 62-17)',
 'Sheet 3 (DO 39-17)',
 'Sheet 3 (DO 078-2018)',
 'Sheet 4  (DO 057-2018)',
 'Sheet 7 (DO 94-15)',
 'Sheet 6 (DO 22-2015)',
 'Sheet 5 (DO

In [ ]:
incomplete = (preprocessed_sheetnames
.query("sheet_name != 'NOTICE'")
.query("sheet_name != 'NOTICE '")
.query("sheet_name != 'Notice'")
.sort_values("vicinity")
.head(3)
)
incomplete
# these 3 have misaligned column, input column nums in parser function?

,revenue_region,RDO,sheet_name,vicinity
37,Revenue Region 5 - Caloocan,RDO No. 27 - Caloocan City,Sheet 7 (DO 041-19,1
4,Revenue Region 5 - Caloocan,RDO No. 24 - Valenzuela City,Sheet 5 (DO 81-15,1
34,Revenue Region 9B - LAQUEMAR (Laguna - Quezon ...,"RDO No. 57 - Biñan City, West Laguna",Sheet 8 (DO 32-19),3


In [ ]:
manual_encode = incomplete.sheet_name.tolist() + missing
len(manual_encode)

55

In [ ]:
manual_encode_df = raw_sheetnames[raw_sheetnames.sheet_name.isin(manual_encode)]
manual_encode_df.to_csv("manual_encode_df.csv",index=False)
manual_encode_df

,filename,sheet_name
1,"bir_zonal_values/RDO No. 106 - Tandag City, Su...",Sheet 4 (DO 057-2018)
2,"bir_zonal_values/RDO No. 106 - Tandag City, Su...",Sheet 3 (DO 12-15)
3,"bir_zonal_values/RDO No. 106 - Tandag City, Su...",Sheet 2 (DO 20-02)
4,"bir_zonal_values/RDO No. 106 - Tandag City, Su...",Sheet 1 (DO 52-95)
2,"bir_zonal_values/RDO No. 115 - Digos City, Dav...",Sheet 3 (DO 71-2015)
...,...,...
0,"bir_zonal_values/RDO No. 96 - Bongao, Tawi-Taw...",Recovered_Sheet1
1,bir_zonal_values/RDO No. 81 - Cebu City North....,Sheet 5 (DO 064-18)
1,bir_zonal_values/RDO No. 26 - Malabon-Navotas.xls,Sheet 7 (DO 94-15)
1,"bir_zonal_values/RDO No. 37 - SanJose, Occiden...",Sheet3 (DO 059-18)


## Manual encode columns for 72

## Preprocess

## Appendix

In [ ]:
# compare sheet count - columns= RDO, sheet_name, raw, processed

sheets = pd.read_excel(excel_file,sheet_name=None)
dfs = []
rdonum = get_rdonum(excel_file)
print(f"Collecting RDO No. {rdonum}")
for k, v in sheets.items():
    df = v
    df2 = add_repeating_column(df,k)
    df3 = unname_columns(df2)
    dfs.append(df3)

In [ ]:
# which sheet names missing
# which sheets with low row count?

In [ ]:
# for region in tqdm(regions):
regions[0]
df = pd.read_csv(f"preprocessed/{region}.csv")


# count rows per sheetname
RDO, sheet_name, row_count
# count sheets per workbook vs raw
RDO, sheet_count
# count how many dates were imputed - just check 1231
RDO, sheet_name, imputed_count
# check start and end of each rdo vs raw

In [ ]:

# Missing because wrong columns
# RDO No. 24 - Valenzuela City	Sheet 5 (DO 81-15


In [ ]:
# next steps
# x add columns rev region, workbook and sheetname on placenames3 using pandas index
# x values are wrong acmac iligan city
# > fill city/municipality columns

In [ ]:
# Next steps:
# x preprocess all files
# view excel headers to check wrong placename detections
# count excel headers to prioritize new rules
#     add new rules for finding placenames
# filter out column names street/subdivision, vicinity

In [ ]:
# create rownum column

# add "usable_row" column

# find rows with barangay,municipality,province info
# output: rownum, area_level (prov/muni/bgy), name

# split by DO No
# if last column has number or last character is *, preserve row (ignores asterisk zonal values)

# just use columns E-H

# output: rownum, street,vicinity,classification,zv_per_sqm




In [ ]:

# get last column with a number
# check preceding columns
# filter columns to text columns
# get 3 left most columns
# right align to street,vicinity,classification

In [ ]:
def test_rowcount(df):
    # need to split by sheet to be able to check rowcount
    df.groupby(sheetname).value_counts